In [1]:
import json
import time
import math
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable

from trainer.asr.trainer import Trainer

from utils import constant
from utils.data_loader import SpectrogramDataset, AudioDataLoader, BucketingSampler
from utils.functions import save_model, load_model, init_transformer_model, init_optimizer
import logging

import sys
import os

usage: ipykernel_launcher.py [-h] [--model MODEL] [--name NAME]
                             [--train-manifest-list TRAIN_MANIFEST_LIST [TRAIN_MANIFEST_LIST ...]]
                             [--valid-manifest-list VALID_MANIFEST_LIST [VALID_MANIFEST_LIST ...]]
                             [--test-manifest-list TEST_MANIFEST_LIST [TEST_MANIFEST_LIST ...]]
                             [--lang-list LANG_LIST [LANG_LIST ...]]
                             [--sample-rate SAMPLE_RATE]
                             [--batch-size BATCH_SIZE]
                             [--num-workers NUM_WORKERS]
                             [--labels-path LABELS_PATH]
                             [--label-smoothing LABEL_SMOOTHING]
                             [--window-size WINDOW_SIZE]
                             [--window-stride WINDOW_STRIDE] [--window WINDOW]
                             [--epochs EPOCHS] [--cuda]
                             [--device-ids DEVICE_IDS [DEVICE_IDS ...]]
                  

SystemExit: 2

/home/aims/anaconda3/envs/aims/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
